In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
# from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import numpy as np
from datetime import datetime
from netCDF4 import Dataset
# from skimage import transform
from plot_picture_function import set_map_ticks,add_Chinese_provinces
from datetime import timedelta
import cartopy.crs as ccrs
import gc

In [2]:
gc.enable()

In [3]:
dataERA5 = xr.open_dataset('E:/ZiYuanPingGu/ERA5/contain_cloudcover/202001.nc')#读取ERA5 20200101整日数据
dataERA5

<xarray.Dataset>
Dimensions:    (longitude: 221, latitude: 161, time: 744)
Coordinates:
  * longitude  (longitude) float32 80.0 80.25 80.5 80.75 ... 134.5 134.8 135.0
  * latitude   (latitude) float32 55.0 54.75 54.5 54.25 ... 15.5 15.25 15.0
  * time       (time) datetime64[ns] 2020-01-01 ... 2020-01-31T23:00:00
Data variables:
    tcc        (time, latitude, longitude) float32 ...
    tciw       (time, latitude, longitude) float32 ...
    tclw       (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-06-16 00:38:35 GMT by grib_to_netcdf-2.24.3: /opt/ecmw...

In [4]:

lon = dataERA5.longitude#经度
lat = dataERA5.latitude#维度
x2, y2 = np.meshgrid(lon, lat)#转化为坐标数据格式

In [25]:
st = '2022-10-01_00:00'
et = '2022-10-31_23:00'
d1 = datetime.strptime(st,'%Y-%m-%d_%H:%M')#type:str--->datetime
d2 = datetime.strptime(et,'%Y-%m-%d_%H:%M')

step = timedelta(minutes = 10)#设置datetime.timedelta类型的步进
seconds = (d2-d1).total_seconds()#转换成分钟   其中d2-d1为datetime.timedelta类型  timedelta只能储存days，seconds，microseconds

files = []
for i in range(0, int(seconds)+1, int(step.total_seconds())):
    if (d1 + timedelta(seconds=i)).minute == 0:
        files.append(d1 + timedelta(seconds=i))

In [26]:
datetime.strftime(files[0],'%Y-%m-%d_%H-%M')

'2022-10-01_00-00'

In [27]:
filesname = [date.strftime('%Y%m/%d/%H/') +
         f'NC_H08_{date.strftime("%Y%m%d_%H%M")}_L2CLP010_FLDK.02401_02401.nc'
         for date in files]

In [28]:
len(filesname)

744

In [29]:
ERA5_data = np.zeros([dataERA5.dims['time'], 3, 161, 221])
JRA55_data = np.zeros([dataERA5.dims['time'], 3, 33, 45])
MERRA2_data = np.zeros([dataERA5.dims['time'], 3, 81, 111])

In [30]:
def Get_H8lonlatindex(inputlon, inputlat):#获取80-135E，55-15N范围内对应的索引
    outputlon = round((inputlon - 80)/0.05)
    outputlat = round(-(inputlat - 60)/0.05)
    return outputlon, outputlat
def transformH8(data1, data2, resolution1, resolution2):#data1是局部数据，data2为全域数据，避免单独处理边缘格点
    num = round((resolution1/resolution2 - 1) / 2)#匹配方框区域格点半边长
    trans_array = np.ones(data1.shape)
    
    # treshold_u = round((resolution1/resolution2 - 1)**2*0.8)#一般情况下的阈值个数（80%的条件）
    
    # dim_array = np.array([np.arange(0, 2*num+1) for i in range(2*num+1)])
    # dimT_array = dim_array.T
    # alpha_array = abs(dim_array-(num)) + abs(dimT_array-(num))
    # max_dist = np.max(alpha_array)
    # total_num = np.sum(max_dist - alpha_array)
    # belta_array = (max_dist - alpha_array)/total_num
#     print(belta_array)

    for i in range(data1.shape[0]):
        for j in range(data1.shape[1]):
            lon = 80 + j*resolution1
            lat = 55 - i*resolution1
#             print('lon:{0}, lat:{1}'.format(lon, lat))
            H8_lonindex, H8_latindex = Get_H8lonlatindex(lon, lat)

            lat_w = H8_latindex-num
            if j == 0:
                temp_array = data2[H8_latindex-num:H8_latindex+num+1, H8_lonindex:H8_lonindex+num+1]
                data_num = len(temp_array.flatten())
                trans_array[i, j] = np.sum(temp_array[~np.isnan(temp_array)])/data_num

            else:         
                temp_array = data2[H8_latindex-num:H8_latindex+num+1, H8_lonindex-num:H8_lonindex+num+1]
                datanum = len(temp_array.flatten())
                trans_array[i, j] = np.mean(temp_array[~np.isnan(temp_array)])
    return trans_array

def transformCLoudH8(data1, data2, resolution1, resolution2):#data1是局部数据，data2为全域数据，避免单独处理边缘格点
    num = round((resolution1/resolution2 - 1) / 2)#匹配方框区域格点半边长
    trans_array = np.zeros(data1.shape)
    
    # treshold_u = round((resolution1/resolution2 - 1)**2*0.8)#一般情况下的阈值个数（80%的条件）
    total_num = (resolution1/resolution2 - 1)**2
    for i in range(data1.shape[0]):
        for j in range(data1.shape[1]):
            lon = 80 + j*resolution1
            lat = 55 - i*resolution1
#             print('lon:{0}, lat:{1}'.format(lon, lat))
            H8_lonindex, H8_latindex = Get_H8lonlatindex(lon, lat)
            if j == 0:
                temp_array = data2[H8_latindex-num:H8_latindex+num+1, H8_lonindex:H8_lonindex+num+1]
                data_num = len(temp_array.flatten())
                # treshold = round(datanum * 0.8)
                # if np.sum(temp_array > 0.8) >= treshold:
                trans_array[i, j] = np.sum(temp_array[~np.isnan(temp_array)])/data_num

            else:
                temp_array = data2[H8_latindex-num:H8_latindex+num+1, H8_lonindex-num:H8_lonindex+num+1]
                data_num = len(temp_array.flatten())
                # if np.sum(temp_array > 0.8) >= treshold_u:
                trans_array[i, j] = trans_array[i, j] = np.sum(temp_array[~np.isnan(temp_array)])/data_num
    return trans_array

In [31]:
import time

In [32]:
def slice_str(str1, st, ed):
    return str1[st:ed]

In [33]:
def Get_Bias(H8filesname):
    print('start!')
    datakuihua = xr.open_dataset('H:/ZiYuanPingGu/hiwimari8_data/hiwamari/' + H8filesname)
    
    
    lon_w = 0#索引，下同
    lon_e = 1100
    lat_n = 100
    lat_s = 900
    
    #验证经纬度是否有错

    #这部分代码会重复利用，目的就是从H8源文件提取想要的数据。
    binary_repr_v = np.vectorize(np.binary_repr)
    slice_str_v = np.vectorize(slice_str)
    qa = datakuihua.QA
    qa_int = qa.data
    qa_int = qa_int.astype(np.int)
    qa_bin = binary_repr_v(qa_int, 16)
    # st = time.time()
    qa65 = slice_str_v(qa_bin, 9, 11)
    qasc = slice_str_v(qa_bin, -5, -3)
    #云水、云冰检测
    
    # qa65 = np.array([qa_bin[i][j][9:11] for i in range(len(qa_bin)) for j in range(len(qa_bin))])
    #云检测
    # qasc = np.array([qa_bin[i][j][-5:-3] for i in range(len(qa_bin)) for j in range(len(qa_bin))])
    # et = time.time()
    # print("IO time cost:", et-st)
    qa65 = qa65.reshape(2401,2401)
    qasc = qasc.reshape(2401,2401)
    qa_wc = np.zeros([2401, 2401])*np.nan#水云数据矩阵初始化
    qa_ic = np.zeros([2401, 2401])*np.nan
    qa_cloud = np.zeros([2401, 2401])
    qa_wc[qa65 == '01'] = 1
    qa_ic[qa65 == '11'] = 1
    qa_cloud[qasc == '11'] = 1
    print('clear ?:', np.all(qa_cloud == 0)) #False就对了
    et = time.time()
    # print("IO time cost:", et-st)
    #匹配空间分辨率
    ct = datakuihua.CLOT.values#[lat_n:lat_s+1:5,lon_w:lon_e+1:5]
    cr = datakuihua.CLER_23.values#[lat_n:lat_s+1:5,lon_w:lon_e+1:5]
    clt = datakuihua.CLTT.values#[lat_n:lat_s+1:5,lon_w:lon_e+1:5]

    #剔除无效值
    cr[cr < -326] = np.nan
    ct[ct < -326] = np.nan
    
    print('cr shape:', cr.shape)
    
    H8lwp = ct * cr * 5/9 * 0.001
    lwp_qa = qa_wc * H8lwp
    #剔除温度大于268华氏摄氏度的格点
    lwp_qa[clt < 268] = np.nan
    lwparray_asERA5 = transformH8(np.zeros([161,221]), lwp_qa, 0.25, 0.05)
    lwparray_asJRA55 = transformH8(np.zeros([33,45]), lwp_qa, 1.25, 0.05)
    lwparray_asMERRA2 = transformH8(np.zeros([81,111]), lwp_qa, 0.5, 0.05)
    
    H8iwp = (ct**(1/0.84))/0.065 * 0.001
    iwp_qa = qa_ic * H8iwp
#     iwp_ori = iwp_qa
    # iwp_qa[clt < 268] = np.nan     
    iwparray_asERA5 = transformH8(np.zeros([161,221]), iwp_qa, 0.25, 0.05)
    iwparray_asJRA55 = transformH8(np.zeros([33,45]), iwp_qa, 1.25, 0.05)
    iwparray_asMERRA2 = transformH8(np.zeros([81,111]), iwp_qa, 0.5, 0.05)
    
    #处理cloud_sc 标签
    scarray_asERA5 = transformCLoudH8(np.zeros([161,221]), qa_cloud, 0.25, 0.05)
    scarray_asJRA55 = transformCLoudH8(np.zeros([33,45]), qa_cloud, 1.25, 0.05)
    scarray_asMERRA2 = transformCLoudH8(np.zeros([81,111]), qa_cloud, 0.5, 0.05)
    
    # et = time.time()
    # print("CPU time cost:", et-st)
    H8_bar = np.array([lwp_qa, iwp_qa, qa_cloud])
    ERA5_bar = np.array([lwparray_asERA5, iwparray_asERA5, scarray_asERA5])
    JRA55_bar = np.array([lwparray_asJRA55, iwparray_asJRA55, scarray_asJRA55])
    MERRA2_bar = np.array([lwparray_asMERRA2, iwparray_asMERRA2, scarray_asMERRA2])
    # print('over!')
    return H8_bar, ERA5_bar, JRA55_bar, MERRA2_bar
    

In [34]:
for fileindex in range(0, len(filesname)):
    try:
        H8_bar, ERA5_bar, JRA55_bar, MERRA2_bar = Get_Bias(filesname[fileindex])
        ERA5_data[fileindex,:,:,:] = ERA5_bar
        JRA55_data[fileindex,:,:,:] = JRA55_bar
        MERRA2_data[fileindex,:,:,:] = MERRA2_bar
    except:
        print("Process *************error:", fileindex)#利用一些输出来查看代码的出错点。
    # print("Process:", fileindex)
        # continue
#     qa_c[:,:,fileindex], H8_lwparray[:,:,fileindex], H8_iwparray[:,:,fileindex] = Get_Bias(filesname[fileindex], fileindex)
    print("Process:", fileindex)

start!


C:\Users\dell\AppData\Local\Temp\ipykernel_33568\1818293985.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  qa_int = qa_int.astype(np.int)


clear ?: False
cr shape: (2401, 2401)
Process: 0
start!
clear ?: False
cr shape: (2401, 2401)
Process: 1
start!
clear ?: False
cr shape: (2401, 2401)
Process: 2
start!
clear ?: False
cr shape: (2401, 2401)
Process: 3
start!
clear ?: False
cr shape: (2401, 2401)
Process: 4
start!
clear ?: False
cr shape: (2401, 2401)
Process: 5
start!
clear ?: False
cr shape: (2401, 2401)
Process: 6
start!
clear ?: False
cr shape: (2401, 2401)
Process: 7
start!
clear ?: False
cr shape: (2401, 2401)
Process: 8
start!
clear ?: False
cr shape: (2401, 2401)
Process: 9
start!
clear ?: False
cr shape: (2401, 2401)
Process: 10
start!
clear ?: False
cr shape: (2401, 2401)
Process: 11
start!
clear ?: True
cr shape: (2401, 2401)
Process: 12
start!
clear ?: True
cr shape: (2401, 2401)
Process: 13
start!
clear ?: True
cr shape: (2401, 2401)
Process: 14
start!
clear ?: True
cr shape: (2401, 2401)
Process: 15
start!
clear ?: True
cr shape: (2401, 2401)
Process: 16
start!
clear ?: False
cr shape: (2401, 2401)
Process:

In [ ]:
# fileindex

In [ ]:
#nc文件不允许有nan数据类型存在
# H8_lwparray[np.isnan(H8_lwparray)] = -327
# H8_iwparray[np.isnan(H8_iwparray)] = -327

In [35]:
ds = xr.Dataset({'ERA5_data': (('time','phase','lat1','lon1'), ERA5_data),
                 'JRA55_data': (('time','phase','lat2','lon2'), JRA55_data),
                 'MERRA2_data': (('time','phase','lat3','lon3'), MERRA2_data)
                })
# ds.attrs['time scale'] = st+'-->'+et
ds.to_netcdf('E:/ZiYuanPingGu/H8_Transform/202210.nc')#储存数据集

In [ ]:
# ds = xr.Dataset({'H8_lwp': (('lat','lon','time'), H8_lwparray),
#                  'H8_iwp': (('lat','lon','time'), H8_iwparray),
#                  'cloud_QA':(('lat','lon','time'), qa_c)
#                 })
# # ds.attrs['time scale'] = st+'-->'+et
# ds.to_netcdf('E:/ZiYuanPingGu/analyze/SCQA829_202010.nc')#储存数据集

In [ ]:
# Plot_2_pic(ERA5lwp_00, H8lwp_00, 'ERA5_202007', 'ERA5_lwp', 'H8_lwp', 'rainbow')

In [ ]:
def plot_pic(dataset1, dataset2, dataset3, dataset4):#画图
    extent = [80, 135, 15, 55]
    levels = np.linspace(0, 0.6, 11)
    proj = ccrs.PlateCarree()#选择投影方式，平面投影
    
    fig = plt.figure(figsize=(8,6.5), dpi=500)
    ax1 = fig.add_subplot(221, projection=proj)#创建一个轴，或者是说主体
    ax1.set_title('(a)H8', fontsize=12, loc='left')
    
    x_ori = np.linspace(80,135,1101)
    y_ori = np.linspace(55,15,801)
    im1 = ax1.contourf(x_ori, y_ori, dataset1, levels=levels, cmap='rainbow', extend='max')
    add_Chinese_provinces(ax1, lw=0.5, ec='gray', fc='none')#后两个参数是设置eagecolor,facecolor,linewigth线宽
    set_map_ticks(ax1, dx=10, dy=10, nx=1, ny=1, labelsize='small')#自定义函数set_map_tick
    ax1.set_extent(extent, crs=proj)

    ax2 = fig.add_subplot(222, projection=proj)#创建一个轴，或者是说主体
    # 设置经纬度刻度.
    set_map_ticks(ax2, dx=10, dy=10, nx=1, ny=1, labelsize='small')#自定义函数set_map_tick
    ax2.set_title('(b)ERA5', fontsize=12, loc='left')

    X_cut = np.linspace(80,135,221)
    Y_cut = np.linspace(55,15,161)
    im2 = ax2.contourf(X_cut, Y_cut, dataset2, levels=levels, cmap = 'rainbow', extend='max')
    add_Chinese_provinces(ax2, lw=0.5, ec='gray', fc='none')#后两个参数是设置eagecolor,facecolor,linewigth线宽
    ax2.set_extent(extent, crs=proj)
    #设置colorbar的位置
    cax2 = fig.add_axes([ax2.get_position().x1 + 0.01, ax2.get_position().y0, 0.02, ax2.get_position().height])
    plt.colorbar(im2, cax=cax2, label='kg·m$^-$$^2$')
    
    ax3 = fig.add_subplot(223, projection=proj)#创建一个轴，或者是说主体
    # 设置经纬度刻度.
    set_map_ticks(ax3, dx=10, dy=10, nx=1, ny=1, labelsize='small')#自定义函数set_map_tick
    ax3.set_title('(c)JRA55', fontsize=12, loc='left')
    
    X_JRA55 = np.linspace(80,135,45)
    Y_JRA55 = np.linspace(55,15,33)
    im3 = ax3.contourf(X_JRA55, Y_JRA55, dataset3, levels=levels, cmap = 'rainbow', extend='max')
    add_Chinese_provinces(ax3, lw=0.5, ec='gray', fc='none')#后两个参数是设置eagecolor,facecolor,linewigth线宽
    ax3.set_extent(extent, crs=proj)
    
    ax4 = fig.add_subplot(224, projection=proj)#创建一个轴，或者是说主体
    # 设置经纬度刻度.
    set_map_ticks(ax4, dx=10, dy=10, nx=1, ny=1, labelsize='small')#自定义函数set_map_tick

    ax4.set_title('(d)MERRA2', fontsize=12, loc='left')

    X_MERRA2 = np.linspace(80,135,111)
    Y_MERRA2 = np.linspace(55,15,81)
    im4 = ax4.contourf(X_MERRA2, Y_MERRA2, dataset4, levels=levels, cmap = 'rainbow', extend='max')
    add_Chinese_provinces(ax4, lw=0.5, ec='gray', fc='none')#后两个参数是设置eagecolor,facecolor,linewigth线宽
    ax4.set_extent(extent, crs=proj)
    #设置colorbar的位置
    cax4 = fig.add_axes([ax4.get_position().x1 + 0.01, ax4.get_position().y0, 0.02, ax4.get_position().height])
#     cax5 = fig.add_axes([ax4.get_position().x1 + 0.03, ax4.get_position().y1+0.01, 0.04, 0.03])
#     cax5.text(0,0.1,'kg·m$^-$$^2$')
#     cax5.spines['top'].set_visible(False)
#     cax5.spines['bottom'].set_visible(False)
#     cax5.spines['left'].set_visible(False)
#     cax5.spines['right'].set_visible(False)
#     cax5.set_xticks([])
#     cax5.set_yticks([])
    plt.colorbar(im4, cax=cax4, label='kg·m$^-$$^2$')

In [ ]:
#绘制单时次的数据，检查是否有误
# lwp_CHN_ori = H8_bar[1][100:901, 0:1101]
# ERA5_CHN_cw = ERA5_data[fileindex,1,:,:]
# JRA55_CHN_cw = JRA55_data[fileindex,1,:,:]
# MERRA2_CHN_cw = MERRA2_data[fileindex,1,:,:]
# plt.close()
# plot_pic(lwp_CHN_ori, ERA5_CHN_cw, JRA55_CHN_cw, MERRA2_CHN_cw)

In [ ]:
#绘制单时次的数据，检查是否有误
lwp_CHN_ori = H8_bar[0][100:901, 0:1101]
ERA5_CHN_cw = ERA5_data[fileindex,0,:,:]
JRA55_CHN_cw = JRA55_data[fileindex,0,:,:]
MERRA2_CHN_cw = MERRA2_data[fileindex,0,:,:]
plt.close()
plot_pic(lwp_CHN_ori, ERA5_CHN_cw, JRA55_CHN_cw, MERRA2_CHN_cw)

In [ ]:
sc_CHN_ori = H8_bar[2][100:901, 0:1101]
ERA5_CHN_sc = ERA5_data[fileindex,2,:,:]
JRA55_CHN_sc = JRA55_data[fileindex,2,:,:]
MERRA2_CHN_sc = MERRA2_data[fileindex,2,:,:]
plt.close()
plot_pic(sc_CHN_ori, ERA5_CHN_sc, JRA55_CHN_sc, MERRA2_CHN_sc)